## GPT4 Vision OCR implementation - Smriti

In [1]:
pip install openai


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Using the API we make requests to the newly released GPT4-V model to recognize the text from the TROCR Evaluation set data and save the responses in a json format in a dictionary.

In [7]:
from openai import OpenAI
from collections import defaultdict
import base64
import requests
import os

# os.chdir("ml-herbarium-grp")
print(os.getcwd())
json_results = defaultdict()
too_big = []
# OpenAI API Key
api_key = key

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
folder_path = "ml-herbarium-data/TROCR_Training/goodfiles/"
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith(('jpg', 'png'))]

for img in image_files:
    if os.stat(img).st_size > 19000000:
        too_big.append(img)
    else:
    # Getting the base64 string
        base64_image = encode_image(img)
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        
        payload = {
            "model": "gpt-4-vision-preview",
            "messages": [
              {
                "role": "user",
                "content": [
                  {
                    "type": "text",
                    "text": "Extract all the text, both typed and handwritten from this image and display it in a JSON format"
                  },
                  {
                    "type": "image_url",
                    "image_url": {
                      "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                  }
                ]
              }
            ],
            "max_tokens": 4096
        }
    
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        json_results[img] = response.json()
    
    # print(response.json())

/projectnb/sparkgrp/ml-herbarium-grp


#### Note : We observed that the image file size was limited to 20MB and so we were not able to pass the higher resolution images (162 of them) to the model. Future steps would involve compressing the larger files to be more size appropriate for GPT4-V. We currently obtain results for 64 images from the evaluation set.

In [14]:
copyjson = json_results.copy()
print(json_results.keys())
for l in list(json_results.keys()):
    if "choices" not in json_results[l].keys():
        del json_results[l]
    else:
        json_results[l] = json_results[l]['choices'][0]['message']['content']

print(len(json_results))    

dict_keys(['ml-herbarium-data/TROCR_Training/goodfiles/1802552799.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1998322454.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/2595747531.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1998413329.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1675940934.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1998540182.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1990825865.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/2236176339.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1999330570.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1998481102.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/2265485412.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/2573054178.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/3341257544.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1455960532.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1998370401.jpg', 'ml-herbarium-data/TROCR_Training/goodfiles/1317726996.jpg', 'ml-herbarium

AttributeError: 'str' object has no attribute 'keys'

In [12]:
print(len(too_big))

162


In [24]:
paths = list(json_results.keys())
folder_path = "ml-herbarium-data/TROCR_Training/goodfiles/"
img_names = [i.replace(folder_path, "") for i in paths]

# print(img_names)
final_dict = defaultdict()

for img in img_names:
    final_dict[img] = json_results[folder_path+img]

print(final_dict.keys())

dict_keys(['1802552799.jpg', '1998322454.jpg', '2595747531.jpg', '1998413329.jpg', '1675940934.jpg', '1998540182.jpg', '1990825865.jpg', '2236176339.jpg', '1999330570.jpg', '1998481102.jpg', '2265485412.jpg', '2573054178.jpg', '3341257544.jpg', '1455960532.jpg', '1998370401.jpg', '1317726996.jpg', '1999314245.jpg', '1929881359.jpg', '1998550976.jpg', '1998465329.jpg', '3341248414.jpg', '1998729261.jpg', '3467354375.jpg', '1998980522.jpg', '1929709919.jpg', '1931124118.jpg', '1455174725.jpg', '1998882868.jpg', '1998590947.jpg', '1702827727.jpg', '437160969.jpg', '1999032409.jpg', '2236597761.jpg', '2595756978.jpg', '1998543749.jpg', '1317746297.jpg', '1998758107.jpg', '1998686142.jpg', '1852143901.jpg', '1999283271.jpg', '2858981761.jpg', '2265566552.jpg', '2859305213.jpg', '1928034398.jpg', '1928479020.jpg', '1999110359.jpg', '2265588715.jpg', '1998994775.jpg', '1702851818.jpg', '1929883118.jpg', '2859205685.jpg', '1852124166.jpg', '1456276626.jpg', '1998387136.jpg', '1999047345.jpg', 

In [62]:
#TEST CODE
# lists = list(json_results.values())
# print(lists[0]['choices'])
# contents = [l['choices'][0]['message']['content'] for l in lists]


[{'message': {'role': 'assistant', 'content': '```json\n{\n  "Image Number": "00427028",\n  "Specimen ID": "1627083",\n  "Herbarium": "UNITED STATES NATIONAL MUSEUM",\n  "Flora": "Flora Hawaiiensis",\n  "Collected by": "C. N. Forbes on Oahu",\n  "Species Name": "Cheirodendron platyphyllum (Hook. & Arn.) Frodin",\n  "Collection Date": "Apr. 26 - May 6 - 1911",\n  "Accession Number": "No. 74318",\n  "Barcode of the Bishop Museum Herbarium": "Image No. 00427028"\n}\n```'}, 'finish_details': {'type': 'stop', 'stop': '<|fim_suffix|>'}, 'index': 0}]


In [63]:
# print(len(contents))

86


In [70]:
# print(contents[1])

```json
{
  "Herbarium Label": {
    "Scientific Name": "Cheirodendron trigynum (Gaud.) A. Heller var. helleri Sherff",
    "Collection Information": "Flora Hawaiianensis. Collected by C. N. Forbes on Oahu.",
    "Location": "Punaluu, Koolau Mts.",
    "Elevation": "Apl. 1-20" + "May 6 - 1914",
    "Collector Number": "XV: 5/18",
    "Barcode": "00427028"
  },
  "Institution Label": {
    "Institution": "UNITED STATES NATIONAL MUSEUM",
    "Specimen Number": "1627083"
  },
  "Imaging Label": {
    "Image Number": "Image No. 00427028"
  }
}
```
Please note that there could be slight inaccuracies in transcription due to the handwriting and the quality of the image. The elevation appears to be a range given with a mixture of dates (April 1-20, May 6, 1914) which is transcribed as given.


#### We save the contents of each image in the form of txt files since there are also comments from the model regarding most of the transcriptions that could be useful while evaluating results.

In [29]:
print(os.getcwd())
for i in final_dict:
    f = open("fall2023/gpt4v-results/"+i.replace("jpg", "txt"), "w")
    f.writelines(final_dict[i])
    f.close()

/projectnb/sparkgrp/ml-herbarium-grp


### To compare with results from Azure Vision, we use a subset of specimen from GBIF (low resolution since Azure vision does not accept the size of specimen in the TROCR eval set), the code is exactly the same as before, only this time on low resolution images.

In [3]:
from openai import OpenAI
from collections import defaultdict
import base64
import requests
import os

# os.chdir("ml-herbarium-grp")
print(os.getcwd())
json_results = defaultdict()
too_big = []
# OpenAI API Key
api_key = key

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
folder_path = "LLM_Specimens/"
image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith(('jpg', 'png'))]

for img in image_files:
    if os.stat(img).st_size > 19000000:
        too_big.append(img)
    else:
    # Getting the base64 string
        base64_image = encode_image(img)
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        
        payload = {
            "model": "gpt-4-vision-preview",
            "messages": [
              {
                "role": "user",
                "content": [
                  {
                    "type": "text",
                    "text": "Extract all the text, both typed and handwritten from this image and display it in a JSON format"
                  },
                  {
                    "type": "image_url",
                    "image_url": {
                      "url": f"data:image/jpeg;base64,{base64_image}"
                    }
                  }
                ]
              }
            ],
            "max_tokens": 4096
        }
    
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        json_results[img] = response.json()
    
    # print(response.json())

/projectnb/sparkgrp/ml-herbarium-grp/fall2023


In [4]:
copyjson = json_results.copy()
print(json_results.keys())
for l in list(json_results.keys()):
    if "choices" not in json_results[l].keys():
        del json_results[l]
    else:
        json_results[l] = json_results[l]['choices'][0]['message']['content']

print(len(json_results))    

dict_keys(['LLM_Specimens/Text_Sample_3.png', 'LLM_Specimens/Mixed_Sample_5.png', 'LLM_Specimens/Text_Sample_11.png', 'LLM_Specimens/Text_Sample_17.png', 'LLM_Specimens/Text_Sample_5.png', 'LLM_Specimens/Mixed_Sample_4.png', 'LLM_Specimens/Mixed_Sample_1.png', 'LLM_Specimens/Text_Sample_1.png', 'LLM_Specimens/Text_Sample_20.png', 'LLM_Specimens/Text_Sample_14.png', 'LLM_Specimens/Text_Sample_19.png', 'LLM_Specimens/Text_Sample_16.png', 'LLM_Specimens/Mixed_Sample_9.png', 'LLM_Specimens/Text_Sample_13.png', 'LLM_Specimens/Text_Sample_12.png', 'LLM_Specimens/Mixed_Sample_6.png', 'LLM_Specimens/Mixed_Sample_2.png', 'LLM_Specimens/Text_Sample_18.png', 'LLM_Specimens/Text_Sample_15.png', 'LLM_Specimens/Mixed_Sample_3.png', 'LLM_Specimens/Text_Sample_7.png', 'LLM_Specimens/Text_Sample_9.png', 'LLM_Specimens/Text_Sample_10.png', 'LLM_Specimens/Text_Sample_6.png', 'LLM_Specimens/Text_Sample_2.png', 'LLM_Specimens/Mixed_Sample_8.png', 'LLM_Specimens/Text_Sample_8.png', 'LLM_Specimens/Mixed_Samp

In [5]:
paths = list(json_results.keys())
folder_path = "LLM_Specimens/"
img_names = [i.replace(folder_path, "") for i in paths]

# print(img_names)
final_dict = defaultdict()

for img in img_names:
    final_dict[img] = json_results[folder_path+img]

print(final_dict.keys())

dict_keys(['Text_Sample_3.png', 'Mixed_Sample_5.png', 'Text_Sample_11.png', 'Text_Sample_17.png', 'Text_Sample_5.png', 'Mixed_Sample_4.png', 'Mixed_Sample_1.png', 'Text_Sample_1.png', 'Text_Sample_20.png', 'Text_Sample_14.png', 'Text_Sample_19.png', 'Text_Sample_16.png', 'Mixed_Sample_9.png', 'Text_Sample_13.png', 'Text_Sample_12.png', 'Mixed_Sample_6.png', 'Mixed_Sample_2.png', 'Text_Sample_18.png', 'Text_Sample_15.png', 'Mixed_Sample_3.png', 'Text_Sample_7.png', 'Text_Sample_9.png', 'Text_Sample_10.png', 'Text_Sample_6.png', 'Text_Sample_2.png', 'Mixed_Sample_8.png', 'Text_Sample_8.png', 'Mixed_Sample_7.png', 'Mixed_Sample_10.png', 'Text_Sample_4.png'])


In [7]:
print(os.getcwd())
for i in final_dict:
    f = open("gpt4v-results-gbif/"+i.replace("png", "txt"), "w")
    f.writelines(final_dict[i])
    f.close()

/projectnb/sparkgrp/ml-herbarium-grp/fall2023


### Observation from the results : 

1. **TROCR Evaluation set** : The labels for each specimen extracted are different according to GPT4-V. Some specimen have labels that are not present in the image itself but GPT4 tries to allocate an appropriate label for the text that it extracts in the json format. This is interesting to note and we will be working on creating uniform labels for every specimen after discussion with the Clients and Professor. There are still inaccuracies in the extracted handwritten texts but we will be able to confirm the accuracy measure only after the subject matter experts go through the results folder. There are also additional comments provided by the model to more accurately assess the text extraction with caution warnings that certain phrases it extracts are from handwritten samples and may not be the best match. Overall promising results found for all the specimen. Folder with extracted text is named **gpt4v-results**

2. **GBIF Evaluation set** : This smaller specimen set was formed to compare results obtained by GPT4-V and Azure Vision. Azure vision cannot handle images of higher size and resolution so we manually extracted specimen (with both plant+label and only label) from the GBIF database to run through both the LLMs. Observations were not as good for GPT4-V, since it could not understand the request we gave it for most of the specimen and was only able to extract text for a few of them. We are experimenting with this a bit more, since it could be an effect of the lower quality images used in this case or an API request issue from their end. Updates on the Azure Vision part are present in the Azure OCR notebook. Folder with extracted text is named **gpt4v-results-gbif**

In [12]:
c=0
l=[]
for i in final_dict:
    if final_dict[i] == "I'm sorry, but I cannot assist with that request." or final_dict[i] == "I'm sorry, but I can't assist with that request.":
        c+=1
        l.append(i)

### The following files could not be processed by GPT4-V

In [13]:
print(c, l)

8 ['Text_Sample_17.png', 'Mixed_Sample_4.png', 'Mixed_Sample_1.png', 'Text_Sample_16.png', 'Mixed_Sample_9.png', 'Text_Sample_15.png', 'Mixed_Sample_3.png', 'Text_Sample_4.png']
